# 1. All the clue functions

### Words in titles of popular movies

In [1432]:
# this isn't super scalable, but these clues are fun and they come up surprisingly often

def movie_clue(word, wordlist='moviewords.txt', clues='movieclues.txt'):
    from nltk.tokenize import word_tokenize    
    
    with open(wordlist, 'r') as g:    
        words = set(g.read().split('\n'))
            
    if word not in words:
        return False
    
    word = word[0].upper() + word[1:]
    
    with open(clues, 'r') as g:
        clues = g.read().split('\n')
        
    for clue in clues:
        if word in word_tokenize(clue):
            return clue.replace(word, '__')

In [1433]:
movie_clue('lord')

'"The __ of the Rings: The Return of the King" (2003 movie)'

### Idioms

In [1438]:
# like movies, this isn't super scalable, but these clues are fun and they come up surprisingly often

def idiom_clue(word, idioms='idioms.txt'):
    with open(idioms, 'r') as g:
        idioms = g.read().split('\n')
    
    for idiom in idioms:
        if word in idiom.split():
            clue = idiom.replace(word, '__')
            return clue[0].upper() + clue[1:]
        
    return False

In [1439]:
idiom_clue('nine')

'Whole __ yards'

### Some definitions

In [1440]:
# definitions for 1000 popular words (scraped from the internet)

def describe(word, file='definitions.txt'):
    with open(file, 'r') as g:
        defs = g.read().split('\n')
            
    for ind,x in enumerate(defs):
        if word == x.split(': ')[0]:
            clue = defs[ind].split(': ')[1]
            return clue[0].upper() + clue[1:]
        
    return False

In [1441]:
describe('aria')

'An elaborate song for solo voice'

### Word pairs

In [1442]:
# from the NYT

def find_seq(word, file='seq_2words_0sw.pkl'):
    
    import pickle
    with open(file, 'rb') as g:
        phrases = pickle.load(g)
            
    lower = word.lower()
    upper = word[0].upper() + word[1:]
    
    # it should be a fairly common phrase
    for phrase in phrases:
        if phrase[1] < 400:
            break
            
        if lower in phrase[0] or upper in phrase[0]:
            clue = phrase[0]
#             print(phrase[1])

            for word in clue:
                if word[0].isupper():
                    return ' '.join([i[0].upper() + i[1:] for i in clue]).replace(lower, '___').replace(upper, '___')
            clue = ' '.join(clue).replace(lower, '___').replace(upper, '___')
            
            return clue[0].upper() + clue[1:]
        
    return False

In [1443]:
find_seq('Obama')

'President ___'

### Word pairs that share a word

In [1456]:
# these kinds of clues are fun, although it's hard to generate reliably good instances

def find_xyz(word, file='seq_2words_0sw.pkl', freqs='wordfreqs.pkl'):
    
    import pickle
    with open(file, 'rb') as g:
        phrases = pickle.load(g)
        
    with open(freqs, 'rb') as g:
        freqs = pickle.load(g)

    answer = []
    
    lower = word.lower()
    upper = word[0].upper() + word[1:]

    swords = set(['would', 'could', 'should', 'I’ve', 'he’s', 'she’s', "I've", "he's", "she's"])
    
    for phrase in phrases:
        if phrase[1] < 10: # change this number to vary results
            continue
        if phrase[0][0] in swords or phrase[0][1] in swords:
            continue
        if '“' in phrase[0][0] or '“' in phrase[0][1] or '"' in phrase[0][0] or '"' in phrase[0][1]:
            continue
        if lower in phrase[0] or upper in phrase[0]:
            other = ' '.join([x for x in phrase[0] if (x != lower and x != upper)])
            answer.append(other)
            if len(answer)==2:
                try:
                    # the model I refer to is my word2vec model
                    if model.similarity(answer[0], answer[1]) < .4: # this is also a good number to play with
                        return 'Word with "%s" or "%s"' % (answer[0], answer[1])
                except:
                    return 'Word with "%s" or "%s"' % (answer[0], answer[1])
                answer.pop()

        
    return False

In [1455]:
find_xyz('leaf')

'Word with "gold" or "fig"'

### WordNet synonyms

In [1267]:
def synset(word, model=model):
    from nltk.corpus import wordnet as wn
    
    syns = []

    try:
        for x in wn.synsets(word):
            for y in x.lemmas():
                syns.append(y.name())
        
        syn = []
        
        for ind,j in enumerate(syns):
            if word[:2] in j or word[0].upper()+word[1:] in j:
                continue
            try:
                if model.similarity(word, j) > 0.4:
                    syn.append(j)
            except:
                continue
                
    except:
        return False

    if len(syn) > 0:
        syn = syn[0].replace('_', ' ')
        return syn[0].upper() + syn[1:]

    return False

In [1460]:
synset('dog')

'Canis familiaris'

### Examples

In [1461]:
# this is pretty neat: WordNet knows category relationships between words

def example(word, freqs='wordfreqs.pkl'):
    from nltk.corpus import wordnet as wn
    
    import pickle
    with open(freqs, 'rb') as g:
        freqs = pickle.load(g)
        
    if freqs[word] < 100:
        return False
         
    ex_d = 1
    ex_u = 1
    try:
        syn = wn.synset('%s.n.01' % word)
    except:
        return False
        
    # examples of the word (i.e., down a category layer)
    try:
        ex_down = [x.name().split('.')[0] for x in syn.hyponyms() if '_' not in x.name().split('.')[0] and word[:3] not in x.name().split('.')[0]][0]
    except:
        ex_d = 0
    
    # what the word is an example of (i.e., up a category layer)
    try:
        ex_up = [x.name().split('.')[0] for x in syn.hypernyms() if '_' not in x.name().split('.')[0] and word[:3] not in x.name().split('.')[0]][0]
    except:
        ex_u = 0
    
    # figure out which direction to return
    if ex_d == 0 and ex_u == 0:
        return False
    elif ex_d == 0 and ex_u == 1:
        return 'Type of ' + ex_up
    elif ex_d == 1 and ex_u == 0:
        return ex_down[0].upper() + ex_down[1:] + ', for one'

    try:
        if model.similarity(ex_up, word) > .4:
            return 'Type of ' + ex_up
    except:
        try:
            if model.similarity(ex_down, word) > .4:
                return ex_down[0].upper() + ex_down[1:] + ', for one'
        except: pass
    
    return False

In [1465]:
example('dog')

'Type of canine'

### word2vec synonyms

In [190]:
# word2vec is fun to play around with, but it can take a long time to train a model

import nltk
import gensim
import os

nltk_path = os.sep.join([os.environ['HOME'], 'nltk_data'])
google_vec_file = '~/GoogleNews-vectors-negative300.bin.gz'

nltk.data.path.insert(0, nltk_path)

In [48]:
model = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True)

In [597]:
# with open('word-file', 'r') as g:
#     words = g.read()
    
# words = set(words.split('\n'))

In [1270]:
def synonyms(answer, words='word-file'):
    
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    
    with open('word-file', 'r') as g:
        words = g.read()
    
    words = set(words.split('\n'))

    similars = model.most_similar(answer)
    
    clue = [x[0] for x in similars if (
        x[0] in words and 
        answer[:2] not in x[0][:2] and 
        x[1]>0.53 and
        ps.stem(x[0]) != ps.stem(answer)
    )]

    for ind,i in enumerate(clue):
        if (i[:2] in [x[:2] for x in clue[:ind]] or
           ps.stem(i) in [ps.stem(x) for x in clue[:ind]]):
            clue.pop(ind)
            
    clue = clue[:2]
    
    try:
        if model.most_similar(positive=clue, topn=1)[0][0] != answer:
            return False
    except:
        return False
    
    clue = ' or '.join(clue[:2])

    if len(clue) < 2:
        return False

    return 'Like ' + clue

In [1469]:
synonyms('creepy')

'Like spooky or freaky'

### WordNet definitions

In [1474]:
def define(word):
    import re
    import inflect
    p = inflect.engine()
    from textblob import TextBlob
    from textblob import Word
    
    from nltk.corpus import wordnet
    try:
        defin = wordnet.synsets(word)[0].definition().split('; ')
    except:
        return False
    
    if word[:3] not in min(defin):
        defin = min(defin)
        defin = re.sub(r'\([^)]*\)', '', defin)
        defin = defin.replace('  ', ' ')
        if defin[0] == ' ':
            defin = defin[1:]
        if defin[-1:] == ' ':
            defin = defin[:-1]
    else:
        return False
    
    if TextBlob(defin[0]).tags[0][1] == 'DT':
        defin = ' '.join(defin.split()[1:])
#         print(defin)
    
    
    # NOTE: this is a very crude way to correctly pluralize a description
    # however, it's more fun than just adding "(plural)" to clues
    # verb tense isn't always correct, either
    if word[-1:] == 's':
        
        singulars = ['A', 'a', 'An', 'an', 'The', 'the']
        pluralclue = []
        first = 0
        
#         print(defin)
                
        for ind,(w, pos) in enumerate(TextBlob(defin).tags):
            if (first == 0 and
                    ('NN' in pos or ('VB' in pos and 
                        TextBlob(defin).tags[ind-1][1] != 'TO' and 
                        w[-1:] != 'd')) and 
                    (ind == len(defin.split())-1 or 'NN' not in TextBlob(defin).tags[ind+1][1]) and
                    #(ind >= len(defin.split())-3 or 'NN' not in TextBlob(defin).tags[ind+3][1]) and 
                    ('ing' not in w and TextBlob(defin).tags[ind-1][0] != 'for')):
                if w[-1:] != 's':
                    pluralclue.append(p.plural(w))
                else:
                    pluralclue.append(w)
#                 print(pos, p.plural(w))
                first = 1
            else:                
                if (('NN' in pos or ('VB' in pos and w[-1:] != 'd' and 
                            TextBlob(defin).tags[ind-1][1] != 'TO' and
                            TextBlob(defin).tags[ind-3][1] != 'TO')) and
                        (ind == len(defin.split())-1 or 'NN' not in TextBlob(defin).tags[ind+1][1]) and
                        #(ind >= len(defin.split())-3 or 'NN' not in TextBlob(defin).tags[ind+3][1]) and 
                        w != 'been' and
                        not ('ing' in w and TextBlob(defin).tags[ind-1][0] != 'for')):
                    if w[-1:] != 's':
                        pluralclue.append(p.plural(w))
                    else:
                        pluralclue.append(w)
#                     print(pos, p.plural(w))
                    first = 1
                else:
                    pluralclue.append(w)
#                     print(pos, w)
        pluralclue = [x for x in pluralclue if x not in singulars]
        pluralclue = ' '.join(pluralclue)
        return pluralclue[0].upper() + pluralclue[1:]
    else:
        return defin[0].upper() + defin[1:]
    
    return False

In [1475]:
define('speed')

'Distance travelled per unit time'

### Previously published clues

In [1272]:
# and if all else fails, just use a clue that's been previously published!
# these are all from the NYT, so we know they're good quality :)

def pull_clue(word, file='nytclues.pkl'):
    import pickle
    from random import randint
    
    with open(file, 'rb') as g:
        cluedic = pickle.load(g)
    
    if word in cluedic:
        
        # a clue with '?' is a play on words, which is fun
        # one of my goals is to be able to write code to generate my own puns
        for x in cluedic[word]:
            if '?' in x:
                return x
            
        # if there's no punny clue for the word, let's just use a random one
        try:
            clue = randint(0, len(cluedic[word])-1)
            return cluedic[word][clue]
        except:
            return False
    
    return False

In [1476]:
pull_clue('black')

'Ravenous?'

### Put all the clue functions together!

In [1477]:
def clue(word):
    
    # this order works reasonably well, combined with the thresholds built into the individual functions
    
    if movie_clue(word):
        return movie_clue(word)
    
    if idiom_clue(word):
        return idiom_clue(word)
    
    if describe(word):
        return describe(word)# + ' (RANDO DEF)'
    
    if find_seq(word):
        return find_seq(word)
    
    if find_xyz(word):
        return find_xyz(word)
    
    if synset(word):
        return synset(word)# + ' (SYNSET)'
    
    if example(word):
        return example(word)# + ' (EXAMPLE)'
    
    if synonyms(word):
        return synonyms(word)# + ' (WORD2VEC)'
    
    if define(word):
        return define(word)# + ' (DEFINITION)'
    
    if pull_clue(word):
        return pull_clue(word)# + ' (PULLED)'
    
    # some words can't be clued yet
    # but I'm working on it!
    return word + ": Figure this out, Zoe..."

# 2. Let's make a grid

In [1479]:
def makegrid(n=15):
    
    # empty grid
    grid = [[' ']*n for i in range(n)]
    
    from random import randint
    
    # this function will find two valid squares to be shaded
    def shade(n=n):
        
        # randomly pick a square
        row = randint(0,(n+1)//2)
        col = randint(0,n-1)
        srow = n-row-1
        scol = n-col-1
        halfway = n+1//2
        
        # we can't pick a square that's already shaded
        if grid[row][col] == '#':
            return shade()
        
        # here's the second square (rotationally symmetric)
        grid[srow][scol] = '#'
        
        # make sure this doesn't create a word shorter than three letters (per NYT rules)        
        if ((row==1 and grid[0][col]==' ') or
                (row==2 and (grid[1][col]==' ' or grid[0][col]==' ')) or
                (row>2 and (grid[row-2][col]=='#' or grid[row-3][col]=='#'))):
            grid[srow][scol] = ' '
            return shade()
        if grid[row+2][col]=='#' or grid[row+3][col]=='#':
            grid[srow][scol] = ' '
            return shade()
        if ((col==1 and grid[row][0]==' ') or
                (col==2 and (grid[row][1]==' ' or grid[row][0]==' ')) or
                (col>2 and (grid[row][col-2]=='#' or grid[row][col-3]=='#'))):
            grid[srow][scol] = ' '
            return shade()
        if ((col==n-2 and grid[row][n-1]==' ') or
                (col==n-3 and (grid[row][n-2]==' ' or grid[row][n-1]==' ')) or
                (col<n-3 and (grid[row][col+2]=='#' or grid[row][col+3]=='#'))):
            grid[n-row-1][n-col-1] = ' '
            return shade()
        
        # if everything works out, we can actually change it now
        grid[row][col] = '#'
        return

    # generate the right number of shaded squares (proportional to size of grid)
    for i in range(0, int((n**2)*.16//2)):
        shade()
        
    grid = [''.join(i) for i in grid]
    
    save it!
    with open('pattern1', 'w') as g:
        g.write('\n'.join(grid) + '\n')
    
    return grid

In [1478]:
makegrid()

['      #   #    ',
 '               ',
 '               ',
 '##       #     ',
 '#      #    ###',
 '      ##   #   ',
 '     #    ##   ',
 '    ##   ##    ',
 '   ##    #     ',
 '   #   ##      ',
 '###    #      #',
 '     #       ##',
 '               ',
 '               ',
 '    #   #      ']

# 3. Fill the grid with words
### (and get a list of the words used)

In [1277]:
def makefill(grid="pattern6", out="randompuzzle.txt", dic="word-file"):
    
    # create the fill using a grid (random by default) and a given dictionary
    from subprocess import call

    with open(out, 'w') as g:
        call(["./cword", grid, dic], stdout=g)
    
    # now we read the fill
    with open(out, 'r') as g:
        fill = g.read()
            
    fill = fill.split('\n')[:-1]
    
    # make csv for d3 purposes
    numbers = ','.join([str(i) for i in range(len(grid))])
    gridlines = []
    for i in grid:
        gridlines.append(','.join([j for j in i]))
    gridlines = '\n'.join(gridlines)
    skeleton = numbers + '\n' + gridlines

    with open('randomskeleton.csv', 'w') as g:
        g.write(skeleton)
    
    # these will be the lists of clues, but we have to work backwards from the fill
    across = []
    down = []
    
    # this is to make a grid for the js representation
    jsgrid = []
    
    # go through and find out which squares should be numbered
    counter = 0
    for rnum, row in enumerate(fill):
        for cnum, char in enumerate(row):
            
            if char=='#':
                jsgrid.append('#')
                continue
            
            # create appropriate numbering (the across and down list numbers must coordinate)
            if cnum==0 or rnum==0 or row[cnum-1]=='#' or fill[rnum-1][cnum]=='#':
                counter += 1
                jsgrid.append(counter)
                
                # record each across word (with corresponding number)
                if cnum==0 or row[cnum-1]=='#':
                    acrossword = row[cnum:].split('#')[0]
                    across.append((counter, acrossword))
                
                # record each down word (with corresponding number)
                if rnum==0 or fill[rnum-1][cnum]=='#':
                    downword = ' '.join(l[cnum] for l in fill[rnum:]).split('#')[0].replace(' ','')
                    down.append((counter, downword))
                    
            if char!='#' and cnum!=0 and rnum!=0 and row[cnum-1]!='#' and fill[rnum-1][cnum]!='#':
                jsgrid.append('')

    
    with open('randomfilllist.txt', 'w') as g:
        filllist = ['Across:'] + ['%s. %s' % (i[0],i[1]) for i in across] + ['Down:'] + ['%s. %s' % (i[0],i[1]) for i in down]
        g.write('\n'.join(filllist))
    
    print([i for i in ''.join(fill)])
#     print(jsgrid)
    print(filllist)
    return #filllist

In [1259]:
makefill()

['F', 'E', 'M', 'U', 'R', '#', 'A', 'C', 'I', 'D', '#', 'S', 'T', 'O', 'P', 'A', 'T', 'O', 'N', 'E', '#', 'C', 'O', 'M', 'A', '#', 'T', 'U', 'B', 'E', 'S', 'C', 'O', 'L', 'D', '#', 'H', 'I', 'P', 'S', '#', 'A', 'L', 'O', 'E', 'T', 'H', 'R', 'E', 'A', 'T', 'E', 'N', '#', 'H', 'O', 'L', 'L', 'E', 'R', '#', '#', '#', 'A', 'C', 'E', 'D', '#', 'R', 'I', 'M', 'L', 'E', 'S', 'S', 'C', 'H', 'A', 'S', 'T', 'E', '#', 'G', 'O', 'N', 'E', '#', '#', '#', '#', 'E', 'A', 'C', 'H', '#', 'S', 'T', 'R', 'U', 'G', 'G', 'L', 'I', 'N', 'G', 'N', 'I', 'N', 'E', 'S', '#', 'E', 'A', 'T', '#', 'A', 'U', 'D', 'I', 'O', 'T', 'R', 'E', 'S', 'P', 'A', 'S', 'S', 'E', 'S', '#', 'M', 'O', 'N', 'O', '#', '#', '#', '#', 'A', 'N', 'T', 'S', '#', 'T', 'A', 'B', 'L', 'E', 'D', 'S', 'O', 'N', 'A', 'T', 'A', 'S', '#', 'S', 'A', 'V', 'E', '#', '#', '#', 'C', 'L', 'O', 'V', 'E', 'R', '#', 'A', 'N', 'T', 'E', 'R', 'O', 'O', 'M', 'R', 'I', 'T', 'E', '#', 'C', 'L', 'U', 'E', '#', 'R', 'I', 'D', 'G', 'E', 'E', 'V', 'E', 'R', '#',

['Across:',
 '1. FEMUR',
 '6. ACID',
 '10. STOP',
 '14. ATONE',
 '15. COMA',
 '16. TUBE',
 '17. SCOLD',
 '18. HIPS',
 '19. ALOE',
 '20. THREATEN',
 '22. HOLLER',
 '24. ACED',
 '25. RIMLESS',
 '26. CHASTE',
 '29. GONE',
 '30. EACH',
 '31. STRUGGLING',
 '37. NINES',
 '39. EAT',
 '40. AUDIO',
 '41. TRESPASSES',
 '44. MONO',
 '45. ANTS',
 '46. TABLED',
 '48. SONATAS',
 '52. SAVE',
 '53. CLOVER',
 '54. ANTEROOM',
 '58. RITE',
 '59. CLUE',
 '61. RIDGE',
 '62. EVER',
 '63. HIRE',
 '64. SNORE',
 '65. WEDS',
 '66. YEAR',
 '67. EGRET',
 'Down:',
 '1. FAST',
 '2. ETCH',
 '3. MOOR',
 '4. UNLEASHES',
 '5. REDACT',
 '6. ACHED',
 '7. COIN',
 '8. IMP',
 '9. DASHING',
 '10. STALL',
 '11. TULLE',
 '12. OBOES',
 '13. PEERS',
 '21. TEES',
 '23. OMEGA',
 '25. ROUTE',
 '26. CENT',
 '27. HAIR',
 '28. ACNE',
 '29. GRASS',
 '32. TESTS',
 '33. LUMBERING',
 '34. IDOL',
 '35. NINE',
 '36. GOOD',
 '38. SPATE',
 '42. ANARCHY',
 '43. STAT',
 '47. AVERSE',
 '48. SCREW',
 '49. OLIVE',
 '50. NOTED',
 '51. AVERS',
 '52.

# 4. Clue the words

In [1274]:
def cluefill(toclue='randomfilllist.txt'):
    with open(toclue, 'r') as g:
        clues = g.read().split('\n')
    
    for ind, line in enumerate(clues):
        if line == 'Across:' or line == 'Down:':
            continue
        
        word = line.split('. ')[1]
        clues[ind] = clues[ind].replace(word, clue(word.lower())) + '(%d letters)' % len(word)
    
    with open('randomclues.txt', 'w') as g:
        g.write('\n'.join(clues))
    
    return clues

In [607]:
cluefill()

['Across:',
 '1. Shield, for example',
 '6. Type of shrub',
 '10. Like nudge or propel(synonyms)',
 '14. S(define)',
 '15. Type of examination',
 '16. "__ the Wild" (2007 movie)',
 '17. A(define)',
 '18. Pimple, for example',
 '19. Type of imitation',
 '20. Like draws or caters(synonyms)',
 '22. Betty Crocker offering(pull clue)',
 '24. A(define)',
 '25. A(define)',
 '26. United ___',
 '29. L(define)',
 '30. Dynamic start?(pull clue)',
 '31. Figure this out, Zoe...',
 '37. T(define)',
 '39. Type of noise',
 '40. S(define)',
 '41. F(define)',
 '44. Type of celebration',
 '45. Type of noisemaker',
 '46. Type of merchant',
 "48. They're blue, in rhyme(pull clue)",
 "52. a person's appearance, manner, or demeanor(describe)",
 '53. I(define)',
 '54. Like neuronal or prefrontal(synonyms)',
 '58. Type of writer',
 '59. Like oolong or coffees(synonyms)',
 '61. Type of gas',
 '62. Figure this out, Zoe...',
 '63. Like eyes or mouths(synonyms)',
 '64. A(define)',
 '65. Word with "nothing" or "we’

# 5. Make a puzzle :)

In [1275]:
def makepuzzle():
    makegrid()
    makefill()
    return cluefill()

In [1386]:
makepuzzle()

['S', 'P', 'E', 'C', 'K', '#', 'L', 'A', 'S', 'T', '#', 'S', 'C', 'A', 'B', 'A', 'L', 'T', 'A', 'R', '#', 'A', 'L', 'O', 'E', '#', 'T', 'A', 'C', 'O', 'M', 'A', 'C', 'R', 'O', '#', 'Y', 'A', 'W', 'N', '#', 'O', 'N', 'T', 'O', 'E', 'N', 'H', 'A', 'N', 'C', 'E', 'S', '#', 'U', 'P', 'R', 'O', 'O', 'T', '#', '#', '#', 'B', 'O', 'O', 'R', '#', 'C', 'O', 'O', 'K', 'E', 'R', 'Y', 'A', 'F', 'F', 'I', 'R', 'M', '#', 'S', 'O', 'U', 'S', '#', '#', '#', '#', 'C', 'L', 'A', 'N', '#', 'B', 'A', 'L', 'U', 'S', 'T', 'R', 'A', 'D', 'E', 'R', 'E', 'S', 'E', 'T', '#', 'G', 'U', 'N', '#', 'S', 'E', 'P', 'I', 'A', 'E', 'X', 'T', 'R', 'O', 'V', 'E', 'R', 'T', 'S', '#', 'F', 'E', 'N', 'S', '#', '#', '#', '#', 'T', 'A', 'N', 'S', '#', 'C', 'O', 'R', 'S', 'E', 'T', 'R', 'E', 'S', 'T', 'A', 'R', 'T', '#', 'F', 'A', 'R', 'E', '#', '#', '#', 'U', 'N', 'H', 'O', 'L', 'Y', '#', 'H', 'A', 'R', 'A', 'S', 'S', 'E', 'S', 'S', 'N', 'O', 'W', '#', 'I', 'D', 'O', 'L', '#', 'C', 'H', 'E', 'A', 'P', 'T', 'U', 'R', 'N', '#',

['Across:',
 '1. Mote (SYNSET)(5 letters)',
 '6. "Indiana Jones and the __ Crusade" (1989 movie)(4 letters)',
 '10. Strikebreaker (SYNSET)(4 letters)',
 '14. Type of table (EXAMPLE)(5 letters)',
 '15. Found chiefly in Africa (DEFINITION)(4 letters)',
 '16. Type of mexican (EXAMPLE)(4 letters)',
 '17. Type of instruction (EXAMPLE)(5 letters)',
 '18. Gape (SYNSET)(4 letters)',
 '19. Word with "hold" or "back"(4 letters)',
 '20. Like strengthens or augments (WORD2VEC)(8 letters)',
 '22. Eradicate (SYNSET)(6 letters)',
 '24. Churl (SYNSET)(4 letters)',
 '25. The act of preparing something by the application of heat (DEFINITION)(7 letters)',
 '26. Confirm (SYNSET)(6 letters)',
 '29. Former French coins of low denominations (DEFINITION)(4 letters)',
 '30. Tribe (SYNSET)(4 letters)',
 '31. Handrail (SYNSET)(10 letters)',
 '37. Type of device (EXAMPLE)(5 letters)',
 '39. ___ violence(3 letters)',
 '40. Burnt sienna (SYNSET)(5 letters)',
 '41. People concerned more with practical realities than